# Import RCMIP2 scenarios

In [1]:
%cd ..

/home/rocky/project/mce


In [2]:
import numpy as np
import pandas as pd
from mce.core import ScenarioBase
from mce.core.forcing import RfAll
from mce.util.io import retrieve_url

## Read RCMIP2 data

In [3]:
fmt = 'rcmip-{}-annual-means-v5-1-0.csv'.format
dfin = {}
for cat_full, cat in {
    'emissions': 'emis',
    'concentrations': 'conc',
    'radiative-forcing': 'rf',
}.items():
    fn = fmt(cat_full)
    path = retrieve_url(
        f'datain/rcmip/{fn}',
        f'https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/{fn}',
    )
    dfin[cat] = pd.read_csv(path, index_col=list(range(7))).rename(columns=int)

[2025-04-09 16:07:29 mce.util.io] INFO:Use local file datain/rcmip/rcmip-emissions-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-emissions-annual-means-v5-1-0.csv on 2025-02-08
[2025-04-09 16:07:30 mce.util.io] INFO:Use local file datain/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-concentrations-annual-means-v5-1-0.csv on 2025-02-09
[2025-04-09 16:07:31 mce.util.io] INFO:Retrieve https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-radiative-forcing-annual-means-v5-1-0.csv
[2025-04-09 16:07:33 mce.util.io] INFO:File datain/rcmip/rcmip-radiative-forcing-annual-means-v5-1-0.csv created


In [4]:
dfin['conc'].index.names

FrozenList(['Model', 'Scenario', 'Region', 'Variable', 'Unit', 'Activity_Id', 'Mip_Era'])

In [5]:
id_vars_1 = ['Scenario', 'Region']
id_vars_2 = ['Model', 'Activity_Id', 'Mip_Era']

## Data conversion for MCE

In [6]:
ds = ScenarioBase()

[2025-04-09 16:09:21 mce.core] INFO:in-memory file opened


### Concentrations

In [9]:
forcing = RfAll()

In [10]:
def conv(x):
    """Variable name conversion
    """
    return (
        x
        .replace('Atmospheric Concentrations|', '')
        .replace('F-Gases|', '')
        .replace('Montreal Gases|', '')
        .replace('CFC|CFC', 'CFC-')
        .replace('HFC|HFC', 'HFC-')
        .replace('PFC|', '')
        .replace('Halon', 'Halon-')
        .replace('HCFC', 'HCFC-')
        .replace('HFC-4310mee', 'HFC-43-10mee')
        .replace('C4F10', 'n-C4F10')
        .replace('C5F12', 'n-C5F12')
        .replace('C6F14', 'n-C6F14')
        .replace('cC4F8', 'c-C4F8')
    )

In [11]:
cat = 'conc'

for k1, v1 in dfin[cat].groupby(id_vars_1):
    kw1 = dict(zip(id_vars_1, k1))
    if kw1['Region'] != 'World':
        continue

    k2, v2 = next(v1.droplevel(id_vars_1).groupby(id_vars_2).__iter__())
    if len(v1) != len(v2):
        raise ValueError('unexpected indexes')

    kw2 = dict(zip(id_vars_2, k2))

    df = (
        v2.droplevel(id_vars_2)
        .rename(conv, level='Variable')
        .T
        .dropna(how='all')
    )

    for x in df.columns.get_level_values(0):
        if x not in forcing.ghgs:
            raise ValueError('unsupported species: {}'.format(x))

    grp = ds.file.create_group('{}/input/{}'.format(kw1['Scenario'], cat))
    
    for k, v in kw2.items():
        grp.attrs[k] = v
    
    dset = grp.create_dataset('time', data=df.index.values)
    dset.attrs['units'] = 'yr'
    
    for k, v in df.items():
        dset = grp.create_dataset(k[0], data=v.values)
        dset.attrs['units'] = k[1]

## Check an example scenario

In [12]:
grp = ds.file['ssp245/input/conc']
list(grp.attrs.items())

[('Activity_Id', 'input4MIPs'),
 ('Mip_Era', 'CMIP6'),
 ('Model', 'MESSAGE-GLOBIOM')]

In [13]:
pd.DataFrame({k: v for k, v in grp.items()}).set_index('time')

,C2F6,C3F8,C7F16,C8F18,CCl4,CF4,CFC-11,CFC-113,CFC-114,CFC-115,...,Halon-1301,Halon-2402,N2O,NF3,SF6,SO2F2,c-C4F8,n-C4F10,n-C5F12,n-C6F14
time,,,,,,,,,,,,,,,,,,,,,
1700,0.00000,0.00000,0.000000,0.000000,0.025,34.049999,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,272.212049,0.00000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000
1701,0.00000,0.00000,0.000000,0.000000,0.025,34.049999,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,272.243047,0.00000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000
1702,0.00000,0.00000,0.000000,0.000000,0.025,34.049999,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,272.266047,0.00000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000
1703,0.00000,0.00000,0.000000,0.000000,0.025,34.049999,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,272.298047,0.00000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000
1704,0.00000,0.00000,0.000000,0.000000,0.025,34.049999,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.0,272.331052,0.00000,0.0000,0.000000,0.00000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,9.59912,1.67188,0.366361,0.176951,0.000,140.825001,0.015106,0.251401,1.05305,3.93032,...,0.003763,0.0,359.840983,2.46639,18.1191,0.000344,3.90381,0.353267,0.179582,0.685107
2497,9.59816,1.67164,0.366239,0.176892,0.000,140.822000,0.014794,0.248462,1.04701,3.92252,...,0.003707,0.0,359.825979,2.46177,18.1134,0.000334,3.90259,0.353196,0.179538,0.684886
2498,9.59720,1.67140,0.366117,0.176833,0.000,140.818998,0.014489,0.245556,1.04100,3.91474,...,0.003651,0.0,359.811981,2.45715,18.1077,0.000325,3.90137,0.353126,0.179494,0.684665


In [14]:
{k: v.attrs['units'] for k, v in grp.items()}

{'C2F6': 'ppt',
 'C3F8': 'ppt',
 'C7F16': 'ppt',
 'C8F18': 'ppt',
 'CCl4': 'ppt',
 'CF4': 'ppt',
 'CFC-11': 'ppt',
 'CFC-113': 'ppt',
 'CFC-114': 'ppt',
 'CFC-115': 'ppt',
 'CFC-12': 'ppt',
 'CH2Cl2': 'ppt',
 'CH3Br': 'ppt',
 'CH3CCl3': 'ppt',
 'CH3Cl': 'ppt',
 'CH4': 'ppb',
 'CHCl3': 'ppt',
 'CO2': 'ppm',
 'HCFC-141b': 'ppt',
 'HCFC-142b': 'ppt',
 'HCFC-22': 'ppt',
 'HFC-125': 'ppt',
 'HFC-134a': 'ppt',
 'HFC-143a': 'ppt',
 'HFC-152a': 'ppt',
 'HFC-227ea': 'ppt',
 'HFC-23': 'ppt',
 'HFC-236fa': 'ppt',
 'HFC-245fa': 'ppt',
 'HFC-32': 'ppt',
 'HFC-365mfc': 'ppt',
 'HFC-43-10mee': 'ppt',
 'Halon-1211': 'ppt',
 'Halon-1301': 'ppt',
 'Halon-2402': 'ppt',
 'N2O': 'ppb',
 'NF3': 'ppt',
 'SF6': 'ppt',
 'SO2F2': 'ppt',
 'c-C4F8': 'ppt',
 'n-C4F10': 'ppt',
 'n-C5F12': 'ppt',
 'n-C6F14': 'ppt',
 'time': 'yr'}

In [15]:
ds.close()

[2025-04-09 16:16:15 mce.core] INFO:in-memory file closed
